In [2]:
!pip install openvino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 3.7 MB/s eta 0:00:0000:0100:01


In [1]:
import sys
sys.path.append("..")
import torch
from models.CQTNet import CQTNet, CQTNetSetlist
import openvino as ov
import os

# Convert CQTNet

In [7]:
CQTNET_MODEL_PATH = "check_points/CQTNet.pth"
CQTNET_OV_MODEL_PATH = "/home/lennon/workspace/CQTNet/check_points/openvino/CQTNET.xml"

In [8]:
cqtnet_model = CQTNet()
cqtnet_model.load_state_dict(torch.load(CQTNET_MODEL_PATH, map_location=torch.device('cpu')))
cqtnet_model.eval()
ov_model_cqtnet = ov.convert_model(cqtnet_model)

In [9]:
if not os.path.isfile(CQTNET_OV_MODEL_PATH):
    ov.save_model(ov_model_cqtnet, CQTNET_OV_MODEL_PATH)
    print("OpenVino model created")
else:
    print("OpenVino model already exists")

OpenVino model created


# Convert CQTNet Finetuned

In [12]:
CQTNET_MODEL_PATH = "check_points/finetuning/latest.pth"
CQTNET_OV_MODEL_PATH = "check_points/openvino/CQTNET_v2_1.xml"

In [3]:
cqtnet_model = CQTNetSetlist() #CQTNet()
cqtnet_model.load_state_dict(torch.load(CQTNET_MODEL_PATH, map_location=torch.device('cpu')))
cqtnet_model.eval()

CQTNetSetlist(
  (features): Sequential(
    (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
    (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    

In [10]:
from openvino.runtime import PartialShape
# input_shape = (1, 1, 128, 64)
# dummy_input = torch.randn(input_shape)
ov_model_cqtnet = ov.convert_model(cqtnet_model, input=(PartialShape([-1, -1, -1, -1]), ov.Type.f32))

In [11]:
ov_model_cqtnet

<Model: 'Model10'
inputs[
<ConstOutput: names[x] shape[?,?,?,?] type: f32>
]
outputs[
<ConstOutput: names[feature.1] shape[?,300] type: f32>
]>

In [6]:
from torchinfo import summary

summary(cqtnet_model)

Layer (type:depth-idx)                   Param #
CQTNetSetlist                            --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       (1,152)
│    └─BatchNorm2d: 2-2                  (64)
│    └─ReLU: 2-3                         --
│    └─Conv2d: 2-4                       (79,872)
│    └─BatchNorm2d: 2-5                  (128)
│    └─ReLU: 2-6                         --
│    └─MaxPool2d: 2-7                    --
│    └─Conv2d: 2-8                       (159,744)
│    └─BatchNorm2d: 2-9                  (128)
│    └─ReLU: 2-10                        --
│    └─Conv2d: 2-11                      (36,864)
│    └─BatchNorm2d: 2-12                 (128)
│    └─ReLU: 2-13                        --
│    └─MaxPool2d: 2-14                   --
│    └─Conv2d: 2-15                      (73,728)
│    └─BatchNorm2d: 2-16                 (256)
│    └─ReLU: 2-17                        --
│    └─Conv2d: 2-18                      (147,456)
│    └─BatchNorm2d: 

In [13]:
if not os.path.isfile(CQTNET_OV_MODEL_PATH):
    ov.save_model(ov_model_cqtnet, CQTNET_OV_MODEL_PATH)
    print("OpenVino model created")
else:
    print("OpenVino model already exists")

OpenVino model created
